In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import keras_tuner as kt
import os


In [2]:

# ====================== Step 1: Load & Clean Dataset ======================
print("Loading dataset...")
df = pd.read_csv("combined_final_data.csv", dtype={"Symbol": "category"})
df['Trade Date'] = pd.to_datetime(df['Trade Date'])
df['Close (Rs.)'] = pd.to_numeric(df['Close (Rs.)'], errors='coerce')
df = df.dropna(subset=['Close (Rs.)'])
df = df.sort_values(by=['Symbol', 'Trade Date'])


Loading dataset...


In [3]:

# ====================== Step 2: Compute Indicators ======================
def compute_indicators(group):
    group['SMA_10'] = group['Close (Rs.)'].rolling(window=10).mean()
    group['EMA_10'] = group['Close (Rs.)'].ewm(span=10).mean()
    group['Momentum'] = group['Close (Rs.)'].diff(10)
    group['Volatility'] = group['Close (Rs.)'].rolling(window=10).std()
    return group

df = df.groupby('Symbol').apply(compute_indicators).reset_index(drop=True)
df = df.dropna()


C:\Users\SachithN\AppData\Local\Temp\ipykernel_12208\2073187434.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = df.groupby('Symbol').apply(compute_indicators).reset_index(drop=True)
C:\Users\SachithN\AppData\Local\Temp\ipykernel_12208\2073187434.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Symbol').apply(compute_indicators).reset_index(drop=True)


In [4]:

# ====================== Step 3: Scale Features ======================
feature_cols = ['SMA_10', 'EMA_10', 'Momentum', 'Volatility']
target_col = 'Close (Rs.)'

feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

df[feature_cols] = feature_scaler.fit_transform(df[feature_cols])
df[[target_col]] = target_scaler.fit_transform(df[[target_col]])



In [5]:
# ====================== Step 4: Create Sequences ======================
def create_sequences(data, feature_cols, target_col, seq_length=60):
    X, y = [], []
    for symbol, group in data.groupby('Symbol'):
        group = group.reset_index(drop=True)
        features = group[feature_cols].values
        target = group[target_col].values

        for i in range(len(group) - seq_length):
            X.append(features[i:i+seq_length])
            y.append(target[i+seq_length])
    return np.array(X), np.array(y)

print("Creating sequences...")
X, y = create_sequences(df, feature_cols, target_col, seq_length=60)



Creating sequences...


C:\Users\SachithN\AppData\Local\Temp\ipykernel_12208\2049676652.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for symbol, group in data.groupby('Symbol'):


In [6]:
# ====================== Step 5: Train-Test Split ======================
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# ====================== Step 6: Build Model for Tuner ======================
def build_model(hp):
    model = Sequential()
    model.add(Bidirectional(
        LSTM(units=hp.Int('units_layer1', min_value=32, max_value=256, step=32), return_sequences=True),
        input_shape=(X_train.shape[1], X_train.shape[2])
    ))
    model.add(Bidirectional(
        LSTM(units=hp.Int('units_layer2', min_value=32, max_value=256, step=32))
    ))
    model.add(Dropout(hp.Float('dropout_rate', 0.1, 0.5, step=0.1)))
    model.add(Dense(1))
    model.compile(
        optimizer=Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss=Huber(),
        metrics=['mae']
    )
    return model

In [7]:

# ====================== Step 7: Tune Hyperparameters ======================
print("Tuning hyperparameters...")
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='keras_tuner_dir',
    project_name='lstm_model'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[stop_early])


Tuning hyperparameters...


d:\Browns\CSE ANALYSER\.conda\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
256               |256               |units_layer1
192               |192               |units_layer2
0.4               |0.4               |dropout_rate
0.01              |0.01              |learning_rate


Epoch 1/10
23286/23286 ━━━━━━━━━━━━━━━━━━━━ 8185s 351ms/step - loss: 0.0040 - mae: 0.0163 - val_loss: 6.1346e-05 - val_mae: 0.0056
Epoch 2/10
17711/23286 ━━━━━━━━━━━━━━━━━━━━ 28:57 312ms/step - loss: 1.0081e-04 - mae: 0.0075

KeyboardInterrupt: 

In [ ]:

# ====================== Step 8: Evaluate Best Model ======================
best_model = tuner.get_best_models(num_models=1)[0]
y_pred_scaled = best_model.predict(X_val)
y_pred_actual = target_scaler.inverse_transform(y_pred_scaled).flatten()
y_val_actual = target_scaler.inverse_transform(y_val.reshape(-1, 1)).flatten()

print("\nModel Evaluation:")
print(f"MSE: {mean_squared_error(y_val_actual, y_pred_actual):.2f}")
print(f"MAE: {mean_absolute_error(y_val_actual, y_pred_actual):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_val_actual, y_pred_actual)):.2f}")
print(f"R2 Score: {r2_score(y_val_actual, y_pred_actual):.4f}")


In [ ]:

# ====================== Step 9: Save Model ======================
model_path = "best_lstm_model.h5"
best_model.save(model_path)
print(f"Model saved as '{model_path}'")